In [9]:
import matplotlib.pyplot as plt
import numpy as np
import math
import matplotlib.animation as animation
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
#from IPython.display import HTML

In [59]:
####################### Useful Constants #########################
G = 6.6743e-11 #m^3kg^-1s^-2; Gravitational Constant
AU = 1.496e11 #m; Astronomical Unit in meters
dt = 24.0*60*60 #s; Seconds in a Day

####################### Defining Classes #########################
class system:
    
    '''
    Solar System Class
    '''
    
    def __init__(self,xdim=32,ydim=32,zdim=32):
        
        '''
        Initializes a Solar System. The Solar System contains Celestial Body objects and simulate
        the movement of the planets using the Planet class methods.
        
        ---Attributes---
        xdim (int): The x size of the Solar System in Astronomical Units (AU)
        ydim (int): The y size of the Solar System in AU
        Bodies (list of Celestial_Body objects): Celestial Body objects in the Solar System
        '''
        
        self.xdim = xdim*AU
        self.ydim = ydim*AU
        self.zdim = zdim*AU
        self.bodies = []
        #self.fig = plt.figure(figsize=(10,10))
        #self.ax = self.fig.add_subplot(projection='3d')
        #self.sct = ax.plot([],[],[],'o', markersize=2)
    
    def add_body(self,body):
        self.bodies.append(body)
        
    def draw(self):
        cmap = [] 
        #self.fig.clear()
        for x in self.bodies:
            cmap.append(x.color)
        print(cmap)
        #creating colormaps does not seem to work
        #new_cmap = LinearSegmentedColormap.from_list('mycmap', cmap)
        print(cmap)
        list_x = []
        list_y = []
        list_z = []
        for x in self.bodies:
            list_x.append(x.pos[0])
            list_y.append(x.pos[1])
            list_z.append(x.pos[2])
        self.ax.scatter(xs=list_x, ys=list_y, zs=list_z, zdir='z', color=cmap)
        #plt.show()
        return(list_x, list_y, list_z)  
    def motion(self, timestep, total_time):
        #timestep in sec, total time in sec
        time_arr = np.arange(0,total_time,timestep)
        grav = lambda mass_one, mass_two, pos_one, pos_two: (6.6743e-11*(mass_one*mass_two))/(pos_one + pos_two)**2
        planets = []
        x_col = []
        y_col = []
        z_col = []
        for curr_time in time_arr:
            for x in self.bodies:
                planets.append(x.name)
                accel_vector = np.array([0,0,0]).astype(np.float64)
                #creating acceleration vectors 
                for y in self.bodies:
                    if y != x:
                        ax = grav(x.mass, y.mass, x.pos[0], y.pos[0])/x.mass
                        ay = grav(x.mass, y.mass, x.pos[1], y.pos[1])/x.mass
                        az = grav(x.mass, y.mass, x.pos[2], y.pos[2])/x.mass
                        accel_vector += np.array([ax,ay,az])
                #changing each planet's velocity according to acceleration (Euler's method)
                x.vel += accel_vector*timestep
                #changing pos
                for x in self.bodies:
                    x.pos += x.vel*timestep
                x_col.append(x.pos[0])
                y_col.append(x.pos[1])
                z_col.append(x.pos[2])
    
        #print(len(x_col), len(y_col), len(z_col), len(time_arr))
        data_dict = {'x':x_col, 'y':y_col, 'z':z_col, 'planet_name':planets}
        data = pd.DataFrame.from_dict(data_dict)
        return data
    def get_positions(self):
        for x in self.bodies:
            print(f'{x.name} at position {x.pos} with velocity {x.vel}')
    def animate(self, num_frames, frames_per_sec):
        def update():
            x_data = []
            y_data = []
            z_data = []
            self.motion(1000/frames_per_sec)
            for b in self.bodies:
                x_data.append(b.pos[0])
                y_data.append(b.pos[1])
                z_data.append(b.pos[2])
            self.ax.scatter.set_data(x_data, y_data, z_data)
        anim = animation.FuncAnimation(self.fig, update, num_frames, interval=1000/frames_per_sec)
        fn = 'plot_3d_scatter_funcanimation'
        anim.save(fn+'.mp4',writer='ffmpeg',fps=frames_per_sec)
        anim.save(fn+'.gif',writer='imagemagick',fps=frames_per_sec)
        plt.rcParams['animation.html'] = 'html5'
        anim
        #anim.save('animation.gif', writer='PillowWriter', fps=2)   
        

In [60]:
class body:
    def __init__(self, init_v=np.array([0,0,0]), init_pos=np.array([0,0,0]), mass=5, name='default', color='blue', size=7):
        self.vel = init_v.astype(np.float64)
        self.pos = init_pos.astype(np.float64)
        self.mass = mass
        self.name = name
        self.color = color
        self.size = size

In [62]:
test_one = body(init_v = np.array([0,0,0]), init_pos=np.array([1000000,2000000,3000000]), color='orange', name='Mars')
test_two = body(init_pos = np.array([-10000000,0.01,0]), color='green', name='Earth')
me = system()
me.add_body(test_one)
me.add_body(test_two)
df = me.motion(timestep = 1, total_time=100)
df.head(50)
#partial = df.iloc[0:len(me.bodies), :]
#fig = px.scatter_3d(partial, x='x', y='y', z='z', color='planet_name')
#fig.show()


,x,y,z,planet_name
0,-10000000.0,0.01,0.000000e+00,Mars
1,-10000000.0,0.01,3.707944e-23,Earth
2,-10000000.0,0.01,7.415889e-23,Mars
3,-10000000.0,0.01,1.483178e-22,Earth
4,-10000000.0,0.01,2.224767e-22,Mars
5,-10000000.0,0.01,3.337150e-22,Earth
6,-10000000.0,0.01,4.449533e-22,Mars
7,-10000000.0,0.01,5.932711e-22,Earth
8,-10000000.0,0.01,7.415889e-22,Mars
9,-10000000.0,0.01,9.269861e-22,Earth
